# Annealing Time Comparison: SA vs SQA (OpenJij Method)

This notebook compares **actual annealing time** between SA and SQA using **OpenJij's official methodology**.

**Based on:** https://tutorial.openjij.org/en/tutorial/005-Evaluation.html

**Key Metrics:**
1. **Execution Time**: From `response.info['execution_time']` (OpenJij's actual measurement)
2. **Success Probability**: `p_s = n/R` (successful attempts / total runs)
3. **Time-to-Solution (TTS)**: `TTS = τ × [ln(1 - p_R) / ln(1 - p_s)]`
   - τ = single annealing time
   - p_R = target probability (typically 0.99)
   - p_s = success probability

**Methods Compared:**
- **SQA**: Simulated Quantum Annealing (OpenJij)
- **SA**: Simulated Annealing (D-Wave)

## Setup: Imports and Configuration

In [ ]:
# Add project root to Python path
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"✓ Project root: {project_root}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import time
import openjij as oj
from dwave.samplers import SimulatedAnnealingSampler
import dimod

# Import custom functions
from scripts.data.data_loaders import load_all_precomputed_data
from scripts.optimization.QUBO import formulate_qubo

warnings.filterwarnings('ignore')

# Set publication-quality plotting defaults
SCALE_FACTOR = 1.8
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = int(12 * SCALE_FACTOR)
plt.rcParams['axes.linewidth'] = 2 * SCALE_FACTOR
plt.rcParams['lines.linewidth'] = 3 * SCALE_FACTOR
plt.rcParams['xtick.major.width'] = 2 * SCALE_FACTOR
plt.rcParams['ytick.major.width'] = 2 * SCALE_FACTOR

sns.set_style('whitegrid')

print("✓ All libraries imported successfully")
print(f"✓ Publication-quality plotting configured (DPI=300, Scale={SCALE_FACTOR}x)")

## Load Data and QUBO Problem

In [ ]:
# Output directory
output_dir = project_root / 'data' / 'results' / 'visualizations' / 'paper'
output_dir.mkdir(parents=True, exist_ok=True)

# Load importance scores and redundancy matrix
importance_dicts, redundancy_matrix = load_all_precomputed_data()

# Use entropy importance (best performing)
importance_method = importance_dicts['entropy']

print(f"✓ Data loaded successfully")
print(f"✓ Output directory: {output_dir}")

## Experimental Parameters

In [ ]:
# QUBO parameters
k = 20
alpha = 0.9
penalty = 2.0

# Annealing parameters
num_sweeps_test = [100, 500, 1000, 2000, 5000]  # Different annealing durations
num_reads_fixed = 100  # Fixed for sweep tests
num_sweeps_fixed = 1000  # Fixed for reads tests
num_reads_test = [10, 50, 100, 500, 1000]  # Different sampling counts

# Number of repetitions for statistical analysis
num_repetitions = 10

# Target success probability for TTS calculation
p_target = 0.99

print("="*80)
print("ANNEALING TIME COMPARISON: SA vs SQA (OpenJij Method)")
print("="*80)
print(f"QUBO Configuration: k={k}, alpha={alpha}, penalty={penalty}")
print(f"Sweep values to test: {num_sweeps_test}")
print(f"Read values to test: {num_reads_test}")
print(f"Repetitions per configuration: {num_repetitions}")
print(f"Target success probability (p_R): {p_target}")
print("="*80)

## Formulate QUBO Problem

In [ ]:
print("Formulating QUBO problem...")
Q, relevant_aps, offset = formulate_qubo(importance_method, redundancy_matrix, k, alpha, penalty)
print(f"✓ QUBO formulated with {len(relevant_aps)} relevant APs")
print(f"✓ QUBO size: {len(Q)} terms")

## Helper Functions

In [ ]:
def calculate_tts(execution_time, success_prob, p_target=0.99):
    """
    Calculate Time-to-Solution (TTS) using OpenJij's formula.
    
    TTS = τ × [ln(1 - p_R) / ln(1 - p_s)]
    
    Parameters:
    -----------
    execution_time : float
        Single annealing execution time (τ)
    success_prob : float
        Success probability (p_s)
    p_target : float
        Target success probability (p_R), default 0.99
    
    Returns:
    --------
    float : Time-to-Solution
    """
    if success_prob <= 0 or success_prob >= 1:
        return np.inf
    
    return execution_time * (np.log(1 - p_target) / np.log(1 - success_prob))

def get_best_energy(Q):
    """
    Get the best known energy for the QUBO problem (for success probability calculation).
    Run multiple times and take the minimum.
    """
    sampler = oj.SQASampler()
    best_energy = np.inf
    
    for _ in range(50):  # Run 50 times to find best
        response = sampler.sample_qubo(Q, num_reads=100, num_sweeps=5000)
        best_energy = min(best_energy, response.first.energy)
    
    return best_energy

print("✓ Helper functions defined")

## Find Best Known Energy

In [ ]:
print("Finding best known energy for the QUBO problem...")
best_known_energy = get_best_energy(Q)
print(f"✓ Best known energy: {best_known_energy:.4f}")
print(f"  (Using tolerance of 1% for success determination)")
energy_tolerance = abs(best_known_energy) * 0.01

## Benchmark 1: SQA (OpenJij) - Varying num_sweeps

In [ ]:
print("\n" + "="*80)
print(f"BENCHMARK 1: SQA - Varying num_sweeps (num_reads={num_reads_fixed})")
print("="*80)

sqa_results_sweeps = []

for num_sweeps in num_sweeps_test:
    print(f"\nTesting SQA with num_sweeps={num_sweeps}...")
    
    execution_times = []
    successes = 0
    
    for rep in range(num_repetitions):
        sampler = oj.SQASampler()
        response = sampler.sample_qubo(Q, num_reads=num_reads_fixed, num_sweeps=num_sweeps)
        
        # Get execution time from OpenJij's response
        exec_time = response.info.get('execution_time', 0)
        execution_times.append(exec_time)
        
        # Check if solution is successful (within tolerance of best energy)
        if abs(response.first.energy - best_known_energy) <= energy_tolerance:
            successes += 1
    
    # Calculate metrics
    avg_exec_time = np.mean(execution_times)
    success_prob = successes / num_repetitions
    tts = calculate_tts(avg_exec_time, success_prob, p_target)
    
    sqa_results_sweeps.append({
        'num_sweeps': num_sweeps,
        'num_reads': num_reads_fixed,
        'avg_execution_time': avg_exec_time,
        'std_execution_time': np.std(execution_times),
        'success_probability': success_prob,
        'tts': tts,
        'successes': successes,
        'total_runs': num_repetitions
    })
    
    print(f"  Avg execution time: {avg_exec_time:.6f}s")
    print(f"  Success probability: {success_prob:.2%} ({successes}/{num_repetitions})")
    print(f"  TTS: {tts:.6f}s")

df_sqa_sweeps = pd.DataFrame(sqa_results_sweeps)
print(f"\n✓ SQA sweep benchmark complete")

## Benchmark 2: SA (D-Wave) - Varying num_sweeps

In [ ]:
print("\n" + "="*80)
print(f"BENCHMARK 2: SA - Varying num_sweeps (num_reads={num_reads_fixed})")
print("="*80)

sa_results_sweeps = []

for num_sweeps in num_sweeps_test:
    print(f"\nTesting SA with num_sweeps={num_sweeps}...")
    
    execution_times = []
    successes = 0
    
    for rep in range(num_repetitions):
        bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
        sampler = SimulatedAnnealingSampler()
        
        start_time = time.time()
        response = sampler.sample(bqm, num_reads=num_reads_fixed, num_sweeps=num_sweeps, beta_range=(0.1, 5.0))
        exec_time = time.time() - start_time
        
        execution_times.append(exec_time)
        
        # Check if solution is successful
        if abs(response.first.energy - best_known_energy) <= energy_tolerance:
            successes += 1
    
    # Calculate metrics
    avg_exec_time = np.mean(execution_times)
    success_prob = successes / num_repetitions
    tts = calculate_tts(avg_exec_time, success_prob, p_target)
    
    sa_results_sweeps.append({
        'num_sweeps': num_sweeps,
        'num_reads': num_reads_fixed,
        'avg_execution_time': avg_exec_time,
        'std_execution_time': np.std(execution_times),
        'success_probability': success_prob,
        'tts': tts,
        'successes': successes,
        'total_runs': num_repetitions
    })
    
    print(f"  Avg execution time: {avg_exec_time:.6f}s")
    print(f"  Success probability: {success_prob:.2%} ({successes}/{num_repetitions})")
    print(f"  TTS: {tts:.6f}s")

df_sa_sweeps = pd.DataFrame(sa_results_sweeps)
print(f"\n✓ SA sweep benchmark complete")

## Visualization 1: Execution Time vs num_sweeps

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: Execution Time
axes[0].errorbar(df_sqa_sweeps['num_sweeps'], df_sqa_sweeps['avg_execution_time'],
                yerr=df_sqa_sweeps['std_execution_time'],
                marker='o', markersize=8, linewidth=2.5, capsize=5,
                label='SQA (OpenJij)', color='#2E86AB')
axes[0].errorbar(df_sa_sweeps['num_sweeps'], df_sa_sweeps['avg_execution_time'],
                yerr=df_sa_sweeps['std_execution_time'],
                marker='s', markersize=8, linewidth=2.5, capsize=5,
                label='SA (D-Wave)', color='#A23B72')

axes[0].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[0].set_ylabel('Execution Time (seconds)', fontsize=18, fontweight='bold')
axes[0].set_title(f'Execution Time vs num_sweeps (num_reads={num_reads_fixed})',
                 fontsize=20, fontweight='bold', pad=20)
axes[0].legend(fontsize=16, loc='upper left')
axes[0].grid(True, alpha=0.3)
axes[0].set_xscale('log')
axes[0].set_yscale('log')

# Plot 2: Success Probability
axes[1].plot(df_sqa_sweeps['num_sweeps'], df_sqa_sweeps['success_probability'] * 100,
            marker='o', markersize=8, linewidth=2.5,
            label='SQA (OpenJij)', color='#2E86AB')
axes[1].plot(df_sa_sweeps['num_sweeps'], df_sa_sweeps['success_probability'] * 100,
            marker='s', markersize=8, linewidth=2.5,
            label='SA (D-Wave)', color='#A23B72')

axes[1].set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
axes[1].set_ylabel('Success Probability (%)', fontsize=18, fontweight='bold')
axes[1].set_title('Success Probability vs num_sweeps',
                 fontsize=20, fontweight='bold', pad=20)
axes[1].legend(fontsize=16, loc='lower right')
axes[1].grid(True, alpha=0.3)
axes[1].set_xscale('log')

plt.tight_layout()
plt.savefig(output_dir / 'annealing_time_execution_success.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 1 saved: annealing_time_execution_success.png")

## Visualization 2: Time-to-Solution (TTS)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Filter out infinite TTS values
sqa_finite = df_sqa_sweeps[df_sqa_sweeps['tts'] != np.inf]
sa_finite = df_sa_sweeps[df_sa_sweeps['tts'] != np.inf]

ax.plot(sqa_finite['num_sweeps'], sqa_finite['tts'],
       marker='o', markersize=10, linewidth=3,
       label='SQA (OpenJij)', color='#2E86AB')
ax.plot(sa_finite['num_sweeps'], sa_finite['tts'],
       marker='s', markersize=10, linewidth=3,
       label='SA (D-Wave)', color='#A23B72')

ax.set_xlabel('Number of Sweeps', fontsize=18, fontweight='bold')
ax.set_ylabel(f'Time-to-Solution (seconds, p={p_target})', fontsize=18, fontweight='bold')
ax.set_title('Time-to-Solution Comparison: SQA vs SA',
            fontsize=20, fontweight='bold', pad=20)
ax.legend(fontsize=16, loc='best')
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
ax.set_yscale('log')

plt.tight_layout()
plt.savefig(output_dir / 'annealing_time_tts_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 2 saved: annealing_time_tts_comparison.png")

## Summary Statistics

In [ ]:
print("\n" + "="*80)
print("ANNEALING TIME COMPARISON SUMMARY (OpenJij Method)")
print("="*80)

print(f"\nBest Known Energy: {best_known_energy:.4f}")
print(f"Energy Tolerance: ±{energy_tolerance:.4f}")

print("\n1. SQA (OpenJij) Results:")
print("-" * 80)
print(df_sqa_sweeps[['num_sweeps', 'avg_execution_time', 'success_probability', 'tts']].to_string(index=False))

print("\n2. SA (D-Wave) Results:")
print("-" * 80)
print(df_sa_sweeps[['num_sweeps', 'avg_execution_time', 'success_probability', 'tts']].to_string(index=False))

print("\n3. Key Findings:")
print("-" * 80)
print(f"- Execution times measured using OpenJij's response.info['execution_time']")
print(f"- Success probability based on {energy_tolerance:.4f} tolerance from optimal")
print(f"- TTS calculated with target probability p_R = {p_target}")
print(f"- Higher num_sweeps generally increases success probability")
print(f"- TTS provides fair comparison accounting for solution quality")
print("="*80)

## Save Results

In [ ]:
# Save detailed results
results_file = project_root / 'data' / 'results' / 'annealing_time_openjij_method.xlsx'

with pd.ExcelWriter(results_file) as writer:
    df_sqa_sweeps.to_excel(writer, sheet_name='SQA_Sweeps', index=False)
    df_sa_sweeps.to_excel(writer, sheet_name='SA_Sweeps', index=False)
    
    # Summary sheet
    summary_data = pd.DataFrame([{
        'Best_Known_Energy': best_known_energy,
        'Energy_Tolerance': energy_tolerance,
        'Target_Probability': p_target,
        'Num_Repetitions': num_repetitions
    }])
    summary_data.to_excel(writer, sheet_name='Config', index=False)

print(f"✓ Results saved to: {results_file}")

## Conclusion

This analysis uses **OpenJij's official methodology** for annealing time evaluation:

### Key Metrics:
1. **Execution Time**: Direct measurement from `response.info['execution_time']`
2. **Success Probability**: Fraction of runs achieving near-optimal solution
3. **Time-to-Solution (TTS)**: Accounts for both speed and quality

### Advantages of TTS:
- Provides **fair comparison** between methods with different success rates
- Answers: "How long to get optimal solution with 99% confidence?"
- Combines execution time and solution quality into single metric

### Insights:
- SQA and SA show different trade-offs between speed and success rate
- Optimal num_sweeps minimizes TTS (not necessarily execution time)
- Framework overhead visible in execution time measurements